In [ ]:
%matplotlib ipympl

# BEER (modulation) McStas data reduction

In [ ]:
import scipp as sc
import scippneutron as scn

from ess.beer import BeerModMcStasWorkflow, BeerModMcStasWorkflowKnownPeaks
from ess.beer.data import mcstas_silicon_medium_resolution, mcstas_duplex, mcstas_quartz, duplex_peaks_array, silicon_peaks_array, quartz_peaks_array
from ess.reduce.nexus.types import Filename, SampleRun
from ess.reduce.time_of_flight.types import DetectorTofData
from ess.beer.types import *

# Default bin edges for our d_hkl histograms
dspacing = sc.linspace('dspacing', 0.8, 2.2, 2000, unit='angstrom')

def ground_truth_peak_positions(p, arr):
    'Helper to display the true peak positions for comparison'
    p.ax.vlines(arr.values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
    return p

## Quick examples using 3 different samples at mode 9 (medium resolution)

### Duplex sample 
#### Known peaks workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing).plot(), duplex_peaks_array())

#### Automatic peak finder workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflow()
wf[ModulationPeriod] = sc.scalar(9e-4, unit='s')
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

### Silicon sample

#### Known peaks workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = silicon_peaks_array()
wf[Filename[SampleRun]] = mcstas_silicon_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing).plot(), silicon_peaks_array())

#### Automatic peak finder workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_silicon_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), silicon_peaks_array())

In [ ]:
wf.compute(DetectorData[SampleRun])['bank2'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

### Quartz sample
#### Known peaks workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing).plot(), quartz_peaks_array())

#### Automatic peak finder workflow

In [ ]:
%%time
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

The quartz sample has many overlapping peaks, which makes the modulation correction difficult:

In [ ]:
wf.compute(DetectorData[SampleRun])['bank2'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

## Duplex sample - Different chopper modes

### Automatic peak finder workflow

#### Mode 7

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(7)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
%%time
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 8

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(8)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 9

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(9)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

In [ ]:
da = da['bank2'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

#### Mode 10

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(10)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

#### Mode 16

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(16)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

### Known peaks workflow

#### Mode 7

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(7)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 8

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(8)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 9

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 10

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(10)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

#### Mode 16

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(16)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), duplex_peaks_array())

## Quartz sample - Different chopper modes

In [ ]:
dspacing = sc.linspace('dspacing', 0.8, 2.5, 2000, unit='angstrom')

### Automatic peak finder workflow

#### Mode 7

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(7)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
%%time
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 8

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(8)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 9

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(9)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

In [ ]:
da = da['bank2'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

#### Mode 10

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(10)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

#### Mode 16

In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_quartz(16)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

### Known peaks workflow

#### Mode 7

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(7)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 8

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(8)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 9

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 10

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(10)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())

#### Mode 16

In [ ]:
wf = BeerModMcStasWorkflowKnownPeaks()
# The quartz_peak_array is very long, restrict it to the dspacing region of interest!
wf[DHKLList] = quartz_peaks_array()[(dspacing[0] <= quartz_peaks_array()) & (quartz_peaks_array() <= dspacing[-1])]
wf[Filename[SampleRun]] = mcstas_quartz(16)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
ground_truth_peak_positions(da.hist(dspacing=dspacing, dim=da.dims).plot(), quartz_peaks_array())